<a href="https://colab.research.google.com/github/Ketan-Suke/-PRODIGY_ML_03-/blob/main/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

imagePaths = []
labels = []
for root,dirnames,filenames in os.walk('/content/drive/MyDrive/INTERNSHIP-prodgy/Cat vs Dog'):
  for filename in filenames:
    if filename.lower().endswith(('png','jpeg','jpg')):
       imagePaths.append(os.path.join(root, filename))

       labels.append(root[74:])

print(len(labels))
print(len(imagePaths))

328
328


In [3]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Define paths to dataset folders
dataset_path = "/content/drive/MyDrive/INTERNSHIP-prodgy/Cat vs Dog"
train_dir = os.path.join(dataset_path, "/content/drive/MyDrive/INTERNSHIP-prodgy/Cat vs Dog/train")
test_dir = os.path.join(dataset_path, "/content/drive/MyDrive/INTERNSHIP-prodgy/Cat vs Dog/test")
val_dir = os.path.join(dataset_path, "/content/drive/MyDrive/INTERNSHIP-prodgy/Cat vs Dog/validation")

# Image preprocessing
img_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(rescale=1.0/255.0)

def load_data(directory):
    generator = datagen.flow_from_directory(
        directory,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )
    return generator

train_generator = load_data(train_dir)
test_generator = load_data(test_dir)
val_generator = load_data(val_dir)

# Load MobileNetV2 for feature extraction
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze the model

def extract_features(generator):
    features, labels = [], []
    for batch_images, batch_labels in generator:
        batch_features = base_model.predict(batch_images)
        batch_features = batch_features.reshape(batch_features.shape[0], -1)
        features.append(batch_features)
        labels.append(batch_labels)
        if len(features) * batch_size >= generator.samples:
            break
    return np.vstack(features), np.hstack(labels)

# Extract features from datasets
X_train, y_train = extract_features(train_generator)
X_test, y_test = extract_features(test_generator)
X_val, y_val = extract_features(val_generator)

# Train SVM classifier
svm = SVC(kernel='linear', C=1.0)
svm.fit(X_train, y_train)

# Predictions and evaluation
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=['Cat', 'Dog']))


Found 86 images belonging to 2 classes.
Found 42 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Test Accuracy: 0.7857142857142857
              precision    recall  f1-score   support

         Cat       0.75      0.86      0.80        21
         Dog       0.83      0.71      0.77        21

    accuracy                           0.79        42
   macro avg       0.79      0.79      0.78        42
weighted avg       0.79      0.79      0.78        42

